# Find threshold for deepseg_sc

This notebook runs an algorithm that finds the best combination of threshold for each contrast, by minimizing the outpout CSA associated with each threshold.

In [1]:
import os
import numpy as np
import pandas as pd
import itertools

In [117]:
folder_csv = '/Volumes/projects/sct_deepseg_threshold/20191009'

thr = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # thresholds to test
contrasts = ['T1', 'T2', 'T2s', 'MTon', 'T1w', 'DWI']  # various contrasts

# Get the number of subjects from the T1 scan, which has all the subjects
df = pd.read_csv(os.path.join(folder_csv, 'csa-T1_0.0.csv'))
n_sub = len(df)

# Read the csv files across contrasts and thresholds
df = {}
for (i_thr, i_contrast) in [(x, y) for x in range(len(thr)) for y in range(len(contrasts))]:
    csv_file = os.path.join(folder_csv, 'csa-{}_{}.csv'.format(contrasts[i_contrast], thr[i_thr]))
    df[i_contrast, i_thr] = pd.read_csv(csv_file)

def retrieve_subject(dataframe):
    """Retrieve subject name from dataframe"""
    return os.path.split(df[0, 0]['Filename'][i_sub])[1].split('_')[0]

# Loop across subjects and populate csa array
csa = []
subjects = []
for i_sub in range(n_sub):
    populate = True
    # Retrieve subject name for the first contrast
    sub = retrieve_subject(df[0, 0])
    # initialize tmp dic to populate contrast and thr info for this subject
    csatmp = {}
    # Loop across threshold and contrast
    for (i_thr, i_contrast) in [(x, y) for x in range(len(thr)) for y in range(len(contrasts))]:
        # Retrieve line corresponding to subject
        dftmp = df[i_contrast, i_thr][df[i_contrast, i_thr]['Filename'].str.contains(sub)]
        if not len(dftmp) == 1:
            print('WARNING: Missing entry for i_sub: {}, thr: {}, contrast: {}'.format(i_sub, thr[i_thr], contrasts[i_contrast]))
            populate = False
            break
        else:
            try:
                # Retrieve the first element of the temporary dataframe (knowing there is only one element)
                # Note: we force it to float, because the presence of "None" in the df converts the whole df to string.
                csatmp[i_contrast, i_thr] = float(dftmp['MEAN(area)'].iloc[0])
            except ValueError as err:
                print('ValueError: {}: i_sub: {}, thr: {}, contrast: {}'.format(err, i_sub, thr[i_thr], contrasts[i_contrast]))
                populate = False
                break
    # If everything went well, populate csa variable
    if populate:
        print("Populate for i_sub: {}".format(i_sub))
        csa.append(csatmp)
        subjects.append(sub)

Populate for i_sub: 0
Populate for i_sub: 1
Populate for i_sub: 2
Populate for i_sub: 3
Populate for i_sub: 4
Populate for i_sub: 5
Populate for i_sub: 6
Populate for i_sub: 7
Populate for i_sub: 8
Populate for i_sub: 9
Populate for i_sub: 10
Populate for i_sub: 11
Populate for i_sub: 12
Populate for i_sub: 13
Populate for i_sub: 14
Populate for i_sub: 15
Populate for i_sub: 16
Populate for i_sub: 20
Populate for i_sub: 21
Populate for i_sub: 23
Populate for i_sub: 24
Populate for i_sub: 29
Populate for i_sub: 30
ValueError: could not convert string to float: 'None': i_sub: 31, thr: 0.0, contrast: T2s
Populate for i_sub: 32
Populate for i_sub: 33
Populate for i_sub: 34
Populate for i_sub: 35
Populate for i_sub: 36
Populate for i_sub: 37
Populate for i_sub: 38
Populate for i_sub: 39
Populate for i_sub: 40
Populate for i_sub: 41
Populate for i_sub: 42
Populate for i_sub: 43
Populate for i_sub: 44
Populate for i_sub: 45
Populate for i_sub: 46
Populate for i_sub: 47
Populate for i_sub: 48


In [119]:
# Update number of subjects based on those who were complete
n_sub = len(csa)

# Loop across subjects
thr_optimal = []
for i_sub in range(n_sub):
    csa_std = []
    i_csa_std = []
    # Loop across all possible permutations of thresholds and contrasts
    for i_thr in list(itertools.permutations(range(len(thr)), len(contrasts))):
        # Compute the STD of CSA across contrasts, for this particular combinations of thresholds
        csa_sub_tmp = [csa[i_sub][x, i_thr[x]] for x in range(len(contrasts))]
        csa_std.append(np.std(csa_sub_tmp))
        i_csa_std.append(i_thr)
#         print(csa_all)
    # Find the minimum across all CSA_STD, for this particular subject
    thr_optmin = [thr[x] for x in i_csa_std[np.argmin(csa_std)]]
    csa_std_optmin = np.std([csa[i_sub][x, thr.index(thr_optmin[x])] for x in range(len(contrasts))])
    csa_std_standard = np.std([csa[i_sub][x, 5] for x in range(len(contrasts))])
#     csa_optmin = [csa[i_sub][x, thr_optmin[x]] for x in range(len(contrasts))]
    thr_optimal.append(thr_optmin)

    thr_optimal_mean = np.mean(thr_optimal, axis=0)
    print("Subject: {} ({}/{})".format(subjects[i_sub], i_sub, n_sub))
    print("  Cumulative average optimal threshold: {}".format(thr_optimal_mean))
    print("  STD(CSA_opt): {} | STD(CSA_standard): {}".format(csa_std_optmin, csa_std_standard))

# Average the threshold per contrast
# print(thr_sub)


Subject: sub-balgrist04 (0/232)
  Cumulative average optimal threshold: [0.8 0.2 0.9 0.4 0.1 0. ]
  STD(CSA_opt): 6.877564042290897 | STD(CSA_standard): 7.888500834822127
Subject: sub-balgrist03 (1/232)
  Cumulative average optimal threshold: [0.8  0.25 0.9  0.3  0.1  0.  ]
  STD(CSA_opt): 8.267512988843206 | STD(CSA_standard): 8.934020495619277
Subject: sub-balgrist05 (2/232)
  Cumulative average optimal threshold: [0.6 0.3 0.9 0.3 0.1 0. ]
  STD(CSA_opt): 7.1173588471981315 | STD(CSA_standard): 8.436954879542158
Subject: sub-balgrist02 (3/232)
  Cumulative average optimal threshold: [0.65  0.275 0.9   0.3   0.1   0.   ]
  STD(CSA_opt): 8.828351581847171 | STD(CSA_standard): 9.78089474464896
Subject: sub-balgrist06 (4/232)
  Cumulative average optimal threshold: [0.68 0.24 0.9  0.28 0.14 0.  ]
  STD(CSA_opt): 8.457245732392058 | STD(CSA_standard): 9.584454460343084
Subject: sub-amu01 (5/232)
  Cumulative average optimal threshold: [0.7        0.23333333 0.9        0.35       0.1333333

Subject: sub-cmrra01 (41/232)
  Cumulative average optimal threshold: [0.67857143 0.38333333 0.89761905 0.33809524 0.09761905 0.02619048]
  STD(CSA_opt): 9.12599272626153 | STD(CSA_standard): 10.084544195546476
Subject: sub-cmrra02 (42/232)
  Cumulative average optimal threshold: [0.68139535 0.39069767 0.89767442 0.33488372 0.09767442 0.0255814 ]
  STD(CSA_opt): 8.216014030413326 | STD(CSA_standard): 9.84523839967609
Subject: sub-cmrra03 (43/232)
  Cumulative average optimal threshold: [0.67045455 0.38863636 0.89772727 0.34545455 0.09772727 0.025     ]
  STD(CSA_opt): 8.58547660173657 | STD(CSA_standard): 9.740006844830079
Subject: sub-cmrra04 (44/232)
  Cumulative average optimal threshold: [0.67333333 0.38444444 0.89777778 0.35333333 0.09777778 0.02444444]
  STD(CSA_opt): 8.867350286487243 | STD(CSA_standard): 10.022077992143867
Subject: sub-cmrra05 (45/232)
  Cumulative average optimal threshold: [0.67608696 0.38043478 0.89782609 0.35217391 0.09782609 0.02391304]
  STD(CSA_opt): 7.5

Subject: sub-milan01 (80/232)
  Cumulative average optimal threshold: [0.69506173 0.34938272 0.89506173 0.34074074 0.10246914 0.01975309]
  STD(CSA_opt): 7.283388199384474 | STD(CSA_standard): 8.500821600951253
Subject: sub-milan02 (81/232)
  Cumulative average optimal threshold: [0.69634146 0.35365854 0.89512195 0.33902439 0.10243902 0.0195122 ]
  STD(CSA_opt): 8.698636336644148 | STD(CSA_standard): 10.020681971742636
Subject: sub-milan03 (82/232)
  Cumulative average optimal threshold: [0.69759036 0.35301205 0.89518072 0.3373494  0.10240964 0.01927711]
  STD(CSA_opt): 8.666215760428878 | STD(CSA_standard): 9.837526801708163
Subject: sub-milan04 (83/232)
  Cumulative average optimal threshold: [0.69880952 0.35714286 0.8952381  0.33809524 0.10119048 0.0202381 ]
  STD(CSA_opt): 9.302655182437825 | STD(CSA_standard): 10.410459025844736
Subject: sub-milan05 (84/232)
  Cumulative average optimal threshold: [0.7        0.35647059 0.89529412 0.33647059 0.10117647 0.02      ]
  STD(CSA_opt): 

Subject: sub-oxfordFmrib06 (119/232)
  Cumulative average optimal threshold: [0.72333333 0.34416667 0.895      0.33333333 0.10916667 0.01666667]
  STD(CSA_opt): 11.147216160343635 | STD(CSA_standard): 12.06230192151994
Subject: sub-oxfordFmrib07 (120/232)
  Cumulative average optimal threshold: [0.72066116 0.34297521 0.89504132 0.33719008 0.10909091 0.01652893]
  STD(CSA_opt): 8.906904482362725 | STD(CSA_standard): 10.550602945507213
Subject: sub-oxfordFmrib08 (121/232)
  Cumulative average optimal threshold: [0.72131148 0.34098361 0.89508197 0.33934426 0.10983607 0.01639344]
  STD(CSA_opt): 7.564271240603619 | STD(CSA_standard): 8.710893938382222
Subject: sub-oxfordFmrib09 (122/232)
  Cumulative average optimal threshold: [0.72276423 0.34390244 0.89430894 0.33821138 0.1097561  0.01626016]
  STD(CSA_opt): 8.899074168623251 | STD(CSA_standard): 9.89530917713357
Subject: sub-oxfordFmrib10 (123/232)
  Cumulative average optimal threshold: [0.7233871  0.34354839 0.89435484 0.33629032 0.110

Subject: sub-sherbrooke05 (158/232)
  Cumulative average optimal threshold: [0.72327044 0.35660377 0.8918239  0.33899371 0.10628931 0.01761006]
  STD(CSA_opt): 7.4960010486473765 | STD(CSA_standard): 8.646988284382402
Subject: sub-sherbrooke06 (159/232)
  Cumulative average optimal threshold: [0.724375 0.355625 0.89125  0.33875  0.10625  0.0175  ]
  STD(CSA_opt): 7.96740202798507 | STD(CSA_standard): 8.910650619129568
Subject: sub-sherbrooke07 (160/232)
  Cumulative average optimal threshold: [0.72546584 0.35465839 0.89068323 0.33850932 0.10621118 0.0173913 ]
  STD(CSA_opt): 6.859790567622552 | STD(CSA_standard): 8.167725737689814
Subject: sub-stanford01 (161/232)
  Cumulative average optimal threshold: [0.72592593 0.35679012 0.89074074 0.33703704 0.10679012 0.01728395]
  STD(CSA_opt): 8.81323208414678 | STD(CSA_standard): 11.718075898370888
Subject: sub-stanford03 (162/232)
  Cumulative average optimal threshold: [0.72392638 0.35582822 0.89079755 0.33680982 0.10674847 0.01717791]
  ST

Subject: sub-tokyoSkyra06 (196/232)
  Cumulative average optimal threshold: [0.73350254 0.35126904 0.89137056 0.33807107 0.11015228 0.01472081]
  STD(CSA_opt): 7.258153508262228 | STD(CSA_standard): 8.157729385371994
Subject: sub-tokyoSkyra07 (197/232)
  Cumulative average optimal threshold: [0.73383838 0.35050505 0.89141414 0.33787879 0.11010101 0.01464646]
  STD(CSA_opt): 7.941271903461155 | STD(CSA_standard): 8.845667935389002
Subject: sub-ubc05 (198/232)
  Cumulative average optimal threshold: [0.73417085 0.35025126 0.89145729 0.33718593 0.11005025 0.01457286]
  STD(CSA_opt): 10.074910148984971 | STD(CSA_standard): 11.227900142180609
Subject: sub-ubc06 (199/232)
  Cumulative average optimal threshold: [0.735  0.3495 0.891  0.3375 0.11   0.0145]
  STD(CSA_opt): 8.368473597654805 | STD(CSA_standard): 9.22019282863704
Subject: sub-ucl02 (200/232)
  Cumulative average optimal threshold: [0.73532338 0.34925373 0.89104478 0.33681592 0.10995025 0.01442786]
  STD(CSA_opt): 9.96826909683901